# Trace Exporting Example

In [ ]:
import os

# This is needed due to zipkin issues related to protobuf.
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# This is needed because examples don't get installed with packages.
import sys

if "../.." not in sys.path:
    sys.path.append("../..")

# Install zipkin python package:
# ! pip install opentelemetry-exporter-zipkin-proto-http

# Start the zipkin docker container:
# ! docker run --rm -d -p 9411:9411 --name zipkin openzipkin/zipkin

# Stop the zipkin docker container:
# ! docker stop $(docker ps -a -q --filter ancestor=openzipkin/zipkin)

In [ ]:
from trulens.core.app.custom import TruCustomApp
from trulens.core.feedback.dummy.endpoint import DummyEndpoint
from trulens.core.tru import Tru
from trulens.dashboard import run_dashboard

from examples.dev.dummy_app.app import DummyApp

Tru().enable_feature("otel_tracing")

# Need to create dummy endpont for the instrumentation of the DummyAPI to
# happen.

DummyEndpoint()

tru = Tru(
    # _experimental_otel_exporter=ZipkinExporter(
    #    endpoint="http://localhost:9411/api/v2/spans"
    # )
)

tru.reset_database()

run_dashboard(force=True, port=8080)

In [ ]:
# Create custom app:
ca = DummyApp()

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

In [ ]:
# Nested context managers.
with ta as recorder1:
    ca.llm.generate("this is the outer memory remember")
    with ta as recorder2:
        ca.memory.remember("this is the inner memory remember")
        ca.respond_to_query("hello")

In [ ]:
from trulens.core.trace import get_tracer

In [ ]:
get_tracer().spans

In [ ]:
recorder1.records